Korean Sentence Splitter(kss) 설치

In [1]:
pip install kss

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 84.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 16.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.7/939.7 kB 68.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 32.9 MB/s eta 0:00:00
  Created wheel for kss: filename=

In [2]:
!file /content/kr3.tsv

/content/kr3.tsv: cannot open `/content/kr3.tsv' (No such file or directory)


원본 데이터셋(utf-8) 불러오기

In [3]:
import pandas as pd
import kss

# 1.
df = pd.read_csv(
    "/content/kr3_utf8.csv",  # tsv 파일 이름
    encoding="utf-8",
    engine="python",
    on_bad_lines="skip",
)

print(df.head())
print(df.columns)
print(len(df))

   Rating                                             Review
0       1  숙성 돼지고기 전문점입니다. 건물 모양 때문에 매장 모양도 좀 특이하지만 쾌적한 편...
1       1  고기가 정말 맛있었어요! 육즙이 가득 있어서 너무 좋았아요 일하시는 분들 너무 친절...
2       1  잡내 없고 깔끔, 담백한 맛의 순댓국이 순댓국을 안 좋아하는 사람들에게도 술술 넘어...
3       1  고기 양이 푸짐해서 특 순대국밥을 시킨 기분이 듭니다 맛도 좋습니다 다만 양념장이 ...
4       1  순댓국 자체는 제가 먹어본 순대국밥집 중에서 Top5 안에는 들어요. 그러나 밥 양...
Index(['Rating', 'Review'], dtype='object')
453798


sentence split labeling template 파일 생성(csv, utf-8)


In [5]:
# 2. Rating이 0/1인 것만 사용 (2는 완전히 제외)
df = df[df["Rating"].isin([0, 1])].copy()

# 3. doc_label을 0/1로 그대로 사용
df["doc_label"] = df["Rating"].astype(int)

# 4. 긍정/부정에서 원하는 개수 샘플링
n_pos = 100
n_neg = 100

pos_df = df[df["Rating"] == 1].sample(
    n=min(n_pos, (df["Rating"] == 1).sum()),
    random_state=42,
)
neg_df = df[df["Rating"] == 0].sample(
    n=min(n_neg, (df["Rating"] == 0).sum()),
    random_state=42,
)

sample_df = pd.concat([pos_df, neg_df], ignore_index=True)

# 5. review_id 부여 (1, 2, 3, ...)
sample_df = sample_df.reset_index(drop=True)
sample_df["review_id"] = sample_df.index + 1

# 6. 문장 단위로 쪼개서 템플릿 만들기
rows = []

for _, row in sample_df.iterrows():
    review_id = int(row["review_id"])
    doc_label = int(row["doc_label"])      # 0 또는 1
    text = str(row["Review"])              # 엑셀에서 Review라는 컬럼명

    # Korean Sentence Splitter(kss)로 한국어 문장 분리
    sentences = kss.split_sentences(text)

    for sent_id, sent in enumerate(sentences, start=1):
        rows.append({
            "review_id": review_id,
            "doc_label": doc_label,        # 0/1
            "sentence_id": sent_id,
            "sentence_text": sent.strip(),
            "sent_label": ""               # 여기다가 나중에 직접 라벨링
        })

sent_df = pd.DataFrame(rows)

# 7.
sent_df.to_csv(
    "/content/kr3_sentence_label.csv",
    index=False,
    encoding="utf-8-sig",               # 엑셀 친화적인 UTF-8 + BOM
)

/usr/local/lib/python3.12/dist-packages/pecab/_tokenizer.py:265: RuntimeWarning: overflow encountered in scalar add
  from_pos_data.costs[idx]
/usr/local/lib/python3.12/dist-packages/pecab/_tokenizer.py:274: RuntimeWarning: overflow encountered in scalar add
  least_cost += word_cost


리뷰 개수 추가(POS/NEG 각 100개씩)

In [6]:
import pandas as pd
import kss

DOC_CSV  = "/content/kr3_utf8.csv"
SENT_CSV = "/content/kr3_sentence_label.csv"

# 1. 원본 KR3 읽기
df = pd.read_csv(DOC_CSV, encoding="utf-8", engine="python", on_bad_lines="skip")
df = df[df["Rating"].isin([0, 1])].copy()
df["doc_label"] = df["Rating"].astype(int)
df = df.reset_index(drop=False).rename(columns={"index": "doc_idx"})  # 각 리뷰의 고유 idx

# 2. 기존 sentence CSV 읽기 (없으면 빈 df)
try:
    sent_df = pd.read_csv(SENT_CSV, encoding="utf-8")
except FileNotFoundError:
    sent_df = pd.DataFrame(columns=[
        "review_id", "doc_label", "sentence_id", "sentence_text", "sent_label", "source_doc_idx"
    ])

if "source_doc_idx" not in sent_df.columns:
    sent_df["source_doc_idx"] = pd.NA

# 3. 이미 쓴 리뷰(doc_idx) 제외
used_doc_idx = set(sent_df["source_doc_idx"].dropna().astype(int).unique())
unused_df = df[~df["doc_idx"].isin(used_doc_idx)].copy()

# 새로 추가할 리뷰 개수
n_new_pos = 100
n_new_neg = 100

pos_df = unused_df[unused_df["Rating"] == 1].sample(
    n=min(n_new_pos, (unused_df["Rating"] == 1).sum()),
    random_state=42,
)
neg_df = unused_df[unused_df["Rating"] == 0].sample(
    n=min(n_new_neg, (unused_df["Rating"] == 0).sum()),
    random_state=42,
)
sample_df = pd.concat([pos_df, neg_df], ignore_index=True)

# 4. review_id는 기존 max 다음부터
if len(sent_df) == 0:
    next_review_id = 1
else:
    next_review_id = int(sent_df["review_id"].max()) + 1

rows = []
for _, row in sample_df.iterrows():
    review_id = next_review_id
    next_review_id += 1

    doc_label = int(row["doc_label"])
    text = str(row["Review"])
    doc_idx = int(row["doc_idx"])

    sentences = kss.split_sentences(text)
    for sent_id, sent in enumerate(sentences, start=1):
        rows.append({
            "review_id": review_id,
            "doc_label": doc_label,
            "sentence_id": sent_id,
            "sentence_text": sent.strip(),
            "sent_label": "",
            "source_doc_idx": doc_idx,
        })

new_sent_df = pd.DataFrame(rows)

# 5. 기존 + 신규 합쳐서 다시 저장 (append 효과)
final_df = pd.concat([sent_df, new_sent_df], ignore_index=True)
final_df.to_csv(SENT_CSV, index=False, encoding="utf-8-sig")

/usr/local/lib/python3.12/dist-packages/pecab/_tokenizer.py:265: RuntimeWarning: overflow encountered in scalar add
  from_pos_data.costs[idx]
/usr/local/lib/python3.12/dist-packages/pecab/_tokenizer.py:274: RuntimeWarning: overflow encountered in scalar add
  least_cost += word_cost
